# Unintentional Hallucination Testing

This notebook tests **edge cases and scenarios** where hallucinations may occur unintentionally.

## Objectives
1. Test knowledge boundaries (cutoff dates, obscure topics)
2. Identify ambiguous queries that trigger fabrication
3. Document statistical claims without sources
4. Establish control baseline with well-known facts

## Test Categories
- Knowledge cutoff issues
- Ambiguous/underspecified queries
- Obscure but real topics
- Statistical/numerical claims
- Speculative future questions
- Control questions (should NOT hallucinate)

In [1]:
# Setup
import sys
sys.path.append('../src')

from agent import HallucinationTestAgent
from database import HallucinationDB
from test_vectors import HallucinationTestVectors
from config import Config
import pandas as pd
from tqdm import tqdm
import time

## Initialize Components

## Validate API Key

Let's verify the API key is loaded and working before running tests.

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI

# Load .env file
env_path = Path('../.env')
load_dotenv(dotenv_path=env_path)

# Get API key
api_key = os.getenv("GROQ_API_KEY")

print("="*60)
print("API KEY VALIDATION")
print("="*60)

# Check if .env file exists
if env_path.exists():
    print(f"✓ .env file found at: {env_path.absolute()}")
else:
    print(f"✗ .env file NOT found at: {env_path.absolute()}")

# Check if API key is loaded
if api_key:
    print(f"✓ API key loaded from .env")
    print(f"  Key starts with: {api_key[:10]}...")
    print(f"  Key length: {len(api_key)} characters")
else:
    print("✗ API key NOT loaded from .env")
    print("  Please check your .env file contains GROQ_API_KEY=your_key_here")

# Test the API key with a simple request
if api_key:
    print("\nTesting API key with Groq API...")
    try:
        client = OpenAI(
            api_key=api_key,
            base_url="https://api.groq.com/openai/v1"
        )
        
        response = client.chat.completions.create(
            model='llama-3.1-8b-instant',
            messages=[{'role': 'user', 'content': 'Say "API key works!"'}],
            max_tokens=10
        )
        
        print(f"✓ API key is VALID!")
        print(f"  Response: {response.choices[0].message.content}")
        print(f"  Tokens used: {response.usage.total_tokens}")
        
    except Exception as e:
        print(f"✗ API key test FAILED!")
        print(f"  Error: {str(e)}")
        print("\n  Possible solutions:")
        print("  1. Get a new API key from https://console.groq.com/keys")
        print("  2. Update .env file with: GROQ_API_KEY=your_new_key")
        print("  3. Restart the kernel to reload .env")

print("="*60)

In [2]:
# Initialize
agent = HallucinationTestAgent()
db = HallucinationDB()

# Create experiment for unintentional tests
unintentional_exp_id = db.create_experiment(
    name="Unintentional Hallucinations - Baseline",
    mitigation_strategy="baseline",
    description="Testing edge cases and knowledge boundaries. Unintentional hallucination scenarios."
)

# Create experiment for control tests
control_exp_id = db.create_experiment(
    name="Control Tests - Baseline",
    mitigation_strategy="baseline",
    description="Well-established facts that should NOT produce hallucinations."
)

print(f"Unintentional tests experiment ID: {unintentional_exp_id}")
print(f"Control tests experiment ID: {control_exp_id}")

Unintentional tests experiment ID: 4
Control tests experiment ID: 5


## Load Test Vectors

In [ ]:
# Get test vectors
unintentional_vectors = HallucinationTestVectors.get_unintentional_vectors()
control_vectors = HallucinationTestVectors.get_control_vectors()

print(f"Unintentional test vectors: {len(unintentional_vectors)}")
print(f"Control test vectors: {len(control_vectors)}")

print("\nUnintentional test categories:")
categories = {}
for vec in unintentional_vectors:
    cat = vec['category']
    categories[cat] = categories.get(cat, 0) + 1

for cat, count in sorted(categories.items()):
    print(f"  - {cat}: {count}")

## Run Unintentional Hallucination Tests

In [ ]:
print("Testing edge cases and knowledge boundaries...\n")

for i, vector in enumerate(tqdm(unintentional_vectors, desc="Unintentional tests")):
    prompt = vector['prompt']
    
    # Query model
    response, metadata = agent.query_baseline(prompt)
    
    # Display
    print("\n" + "="*80)
    print(f"Test {i+1}/{len(unintentional_vectors)}")
    print(f"Category: {vector['category']}")
    print(f"Description: {vector['description']}")
    print(f"\nPrompt: {prompt}")
    print(f"\nResponse:\n{response}")
    print("="*80)
    
    # Manual annotation needed for these (expected_hallucination may be None)
    # For automated run, default to False if None
    expected = vector.get('expected_hallucination')
    is_hallucination = False if expected is None else expected
    
    # Uncomment for manual review:
    # annotation = input("\nDid the model hallucinate? (y/n/u for uncertain): ").strip().lower()
    # is_hallucination = annotation == 'y'
    
    # Determine hallucination type
    hallucination_type = vector['category'] if is_hallucination else 'none'
    
    # Log to database
    db.log_test(
        experiment_id=unintentional_exp_id,
        prompt_text=prompt,
        response_text=response,
        is_hallucination=is_hallucination,
        prompt_category=vector['category'],
        vector_type='unintentional',
        expected_hallucination=vector.get('expected_hallucination'),
        hallucination_type=hallucination_type,
        severity=vector.get('severity', 'low'),
        description=vector['description'],
        response_time_ms=metadata.get('response_time_ms', 0),
        tokens_used=metadata.get('tokens_used', 0)
    )
    
    time.sleep(1)  # Rate limiting

print("\n✓ Unintentional hallucination tests complete!")

## Run Control Tests

These are well-established facts. The model should NOT hallucinate on these.

In [ ]:
print("Testing control questions (should NOT hallucinate)...\n")

for i, vector in enumerate(tqdm(control_vectors, desc="Control tests")):
    prompt = vector['prompt']
    
    # Query model
    response, metadata = agent.query_baseline(prompt)
    
    # Display
    print("\n" + "="*80)
    print(f"Control Test {i+1}/{len(control_vectors)}")
    print(f"\nPrompt: {prompt}")
    print(f"\nResponse:\n{response}")
    print("="*80)
    
    # These should be False (no hallucination expected)
    is_hallucination = False
    
    # Uncomment to verify:
    # check = input("\nDid it hallucinate? (y/n): ").strip().lower()
    # is_hallucination = check == 'y'
    
    # Log to database
    db.log_test(
        experiment_id=control_exp_id,
        prompt_text=prompt,
        response_text=response,
        is_hallucination=is_hallucination,
        prompt_category='control',
        vector_type='control',
        expected_hallucination=False,
        hallucination_type='none',
        severity='low',
        description=vector['description'],
        response_time_ms=metadata.get('response_time_ms', 0),
        tokens_used=metadata.get('tokens_used', 0)
    )
    
    time.sleep(1)

print("\n✓ Control tests complete!")

## Analyze Results

In [ ]:
# Unintentional results
df_unintentional = db.get_experiment_results(unintentional_exp_id)

print("Unintentional Hallucination Results")
print("="*50)
print(f"Total tests: {len(df_unintentional)}")
print(f"Hallucinations: {df_unintentional['is_hallucination'].sum()}")
print(f"Hallucination rate: {df_unintentional['is_hallucination'].mean()*100:.1f}%")

print("\nBy category:")
category_stats = df_unintentional.groupby('prompt_category')['is_hallucination'].agg(['count', 'sum', 'mean'])
category_stats.columns = ['Total', 'Hallucinations', 'Rate']
print(category_stats.round(3))

In [ ]:
# Control results
df_control = db.get_experiment_results(control_exp_id)

print("Control Test Results (should be 0% hallucination)")
print("="*50)
print(f"Total tests: {len(df_control)}")
print(f"Hallucinations: {df_control['is_hallucination'].sum()}")
print(f"Hallucination rate: {df_control['is_hallucination'].mean()*100:.1f}%")

if df_control['is_hallucination'].sum() > 0:
    print("\n⚠️  WARNING: Model hallucinated on control questions!")
    print("These are well-known facts. Review the responses.")
else:
    print("\n✓ Good: No hallucinations on control questions")

## Export Results

In [ ]:
# Export both experiments
unintentional_path = db.export_to_csv(unintentional_exp_id)
control_path = db.export_to_csv(control_exp_id)

print(f"Unintentional results: {unintentional_path}")
print(f"Control results: {control_path}")

## Key Observations

**Document your findings:**

1. **Knowledge Boundaries:**
   - How does the model handle questions beyond its knowledge cutoff?
   - Does it admit uncertainty or fabricate?

2. **Ambiguous Queries:**
   - Does the model invent specifics for vague questions?
   - Does it ask for clarification?

3. **Statistical Claims:**
   - Does it cite specific numbers without sources?
   - How confident does it sound?

4. **Control Performance:**
   - Did any well-known facts get incorrect responses?

**Your notes:**
- 
- 
- 

## Next Steps

Proceed to **03_comparative_analysis.ipynb** to test how mitigation strategies (RAG, Constitutional AI, Chain-of-Thought) perform on these same prompts.

In [ ]:
db.close()